In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import normalized_mutual_info_score
from sklearn.feature_selection import mutual_info_classif
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.naive_bayes import MultinomialNB
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentimental-analysis-for-tweets/sentiment_tweets3.csv


In [2]:
data = pd.read_csv("/kaggle/input/sentimental-analysis-for-tweets/sentiment_tweets3.csv")
data

,Index,message to examine,label (depression result)
0,106,just had a real good moment. i missssssssss hi...,0
1,217,is reading manga http://plurk.com/p/mzp1e,0
2,220,@comeagainjen http://twitpic.com/2y2lx - http:...,0
3,288,@lapcat Need to send 'em to my accountant tomo...,0
4,540,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0
...,...,...,...
10309,802309,No Depression by G Herbo is my mood from now o...,1
10310,802310,What do you do when depression succumbs the br...,1
10311,802311,Ketamine Nasal Spray Shows Promise Against Dep...,1
10312,802312,dont mistake a bad day with depression! everyo...,1


In [3]:
data = data.rename(columns={"message to examine": "text", "label (depression result)": "target"})
data

,Index,text,target
0,106,just had a real good moment. i missssssssss hi...,0
1,217,is reading manga http://plurk.com/p/mzp1e,0
2,220,@comeagainjen http://twitpic.com/2y2lx - http:...,0
3,288,@lapcat Need to send 'em to my accountant tomo...,0
4,540,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0
...,...,...,...
10309,802309,No Depression by G Herbo is my mood from now o...,1
10310,802310,What do you do when depression succumbs the br...,1
10311,802311,Ketamine Nasal Spray Shows Promise Against Dep...,1
10312,802312,dont mistake a bad day with depression! everyo...,1


In [4]:
nltk.download('stopwords')
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))
def preprocess_tweet(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r'http\S+', '', tweet)
    tweet = re.sub(r'@[\w]+', '', tweet)
    tweet = re.sub(r'#\w+', '', tweet)
    tweet = re.sub(r'[^a-záéíóúüñ\s]', '', tweet)
    words = tweet.split()
    words = [stemmer.stem(word) for word in words if word not in stop_words]
    processed_tweet = ' '.join(words)
    return processed_tweet

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
data["processed_text"] = data["text"].apply(preprocess_tweet)
data

,Index,text,target,processed_text
0,106,just had a real good moment. i missssssssss hi...,0,real good moment missssssssss much
1,217,is reading manga http://plurk.com/p/mzp1e,0,read manga
2,220,@comeagainjen http://twitpic.com/2y2lx - http:...,0,
3,288,@lapcat Need to send 'em to my accountant tomo...,0,need send em account tomorrow oddli wasnt even...
4,540,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0,add myspac myspacecomlookthund
...,...,...,...,...
10309,802309,No Depression by G Herbo is my mood from now o...,1,depress g herbo mood im done stress peopl dont...
10310,802310,What do you do when depression succumbs the br...,1,depress succumb brain make feel like youll nev...
10311,802311,Ketamine Nasal Spray Shows Promise Against Dep...,1,ketamin nasal spray show promis depress suicid
10312,802312,dont mistake a bad day with depression! everyo...,1,dont mistak bad day depress everyon em


In [6]:
vectorizer = CountVectorizer(max_features = 10000)
vectorizer.fit(data["processed_text"])
X_train, X_test, y_train, y_test = train_test_split(data[["processed_text"]], data["target"], test_size=0.25, random_state=42)
X_train_counts = vectorizer.transform(X_train["processed_text"])
X_test_counts = vectorizer.transform(X_test["processed_text"])
X_train_counts

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 57293 stored elements and shape (7735, 10000)>

In [7]:
mnb = MultinomialNB()
mnb.fit(X_train_counts, y_train)
y_pred_mnb = mnb.predict(X_test_counts)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_mnb))

              precision    recall  f1-score   support

           0       0.99      0.97      0.98      2011
           1       0.90      0.96      0.93       568

    accuracy                           0.97      2579
   macro avg       0.94      0.96      0.95      2579
weighted avg       0.97      0.97      0.97      2579



In [8]:
#Palabras que aportan información
mi = pd.Series(mutual_info_classif(X_train_counts, y_train))
mi = mi.sort_values(ascending=False)[0:5000]
vocab = pd.Series(vectorizer.vocabulary_.keys())
vocab_mi = vocab.iloc[mi.index]
vocab_mi.head(50)

2288                   warbler
466                        dog
2876                    marvel
4681                     faint
3152                    strike
2017                     sheff
5236                      stat
3974                       tgi
7964    pictwittercomyyweoaaua
7953                  hopeless
7895                    crappp
3947                     gaeul
2366                       bam
2878                  mushroom
1946                     booti
7870                   friggin
3269                   cashier
2489                    pathet
8294                  realcant
6672                       wor
4468     bekahampchrisampswazi
3083                    intens
374                       aint
6839                    remain
1399                    namast
4193                      ambl
2576                   biggest
8106                  postgrad
3873                    stream
4661                      ahhh
9104                 dysphoria
7555                     rehab
1533    

In [9]:
#Palabras que no aportan información
mi = mi.sort_values(ascending=True)[0:50]
vocab_mi_bad = vocab.iloc[mi.index]
vocab_mi_bad

6995                  adrienn
1593                cheesecak
8901                   trisha
7085                sleepback
7210              photographi
7058               coughcough
7230                    pimpl
7227                  shatter
7197                 beforeit
7198              welcomedont
1698                   drinki
7200                    soror
7201                   beleiv
1729                     fuel
7202                  baccara
533                    festiv
1666                  teacher
1594                  factori
7224                   skitch
1602                  omggggg
1742                     feck
7214                    purpl
7216                   ridden
8903               dreambuild
8988    pictwittercomianyoaqc
545                     fresh
8989    pictwittercommkpapsrf
7220                 weeeeeed
1739                      pit
1597                woooooooo
7060                    venit
1596                     tmrw
1595                     prom
1598      

In [10]:
cv = CountVectorizer(vocabulary = vocab_mi.values)
X_train_counts = cv.transform(X_train["processed_text"])
X_test_counts = cv.transform(X_test["processed_text"])
X_train_counts

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 28782 stored elements and shape (7735, 5000)>

In [11]:
mnb = MultinomialNB()
mnb.fit(X_train_counts, y_train)
y_pred_mnb = mnb.predict(X_test_counts)
print(classification_report(y_test, y_pred_mnb))

              precision    recall  f1-score   support

           0       0.98      0.97      0.98      2011
           1       0.90      0.94      0.92       568

    accuracy                           0.96      2579
   macro avg       0.94      0.95      0.95      2579
weighted avg       0.96      0.96      0.96      2579

